In [9]:
import requests
from bs4 import BeautifulSoup


def scrapetle():
    url = 'http://celestrak.com/NORAD/elements/stations.txt'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "lxml")
    mytle=(soup.text[:168])
    lines = mytle.splitlines()
    return (lines)

def tletodict(lines):
    l1=lines[1].split()
    l2=lines[2].split()

    satellite_number=l1[1] 

    isstle={}

    tempvar =l1[3]
    # getting keplerian elements from first line of TLE
    isstle['satellite_number'] =l1[1]

    isstle['epoch_year'] =int('20'+tempvar[0:2])
    isstle['epoch']=float(tempvar[2:12])
    isstle['first_time_deriv_mean_motion']=float('0'+l1[4])
    tempvar=l1[5]
    isstle['second_time_deriv_mean_motion']=float('0.'+tempvar[0:5])
    tempvar=l1[6]
    isstle['bstar_drag']    =float('0.'+tempvar[0:5])
    zeronum=l1[7]
    tempvar=l1[8]
    isstle['element_Set_no']=int(tempvar[0:3])
    # getting keplerian elements from second line of TLE
    isstle['inclination']    =float(l2[2])
    isstle['right_ascension_ascending_node']  =  float(l2[3])
    # todo: fix for when the values are negative with an assumed decimal point
    # todo: the appended -0
    isstle['eccentricity']    = float("0."+l2[4])
    isstle['argument_perigee']    =float(l2[5])
    isstle['mean_anomaly']    =float(l2[6])
    tempvar=lines[2]
    tempvar=tempvar[53:69]
    isstle['mean_motion']   =float(tempvar[0:11])
    print (tempvar)
    isstle['revolution_number_at_epoch'] =int(tempvar[11:15])
    return isstle


In [10]:
lines=scrapetle()

print (lines[1])
print (lines[2])


isstle= tletodict(lines)


import pprint

pp = pprint.PrettyPrinter(indent=4)

pp.pprint(isstle)

1 25544U 98067A   16159.20072255  .00002935  00000-0  50420-4 0  9999
2 25544  51.6430 107.8400 0001564 148.1423 349.8248 15.54761605  3438
5.54761605  3438
{   'argument_perigee': 148.1423,
    'bstar_drag': 0.5042,
    'eccentricity': 0.0001564,
    'element_Set_no': 999,
    'epoch': 159.200722,
    'epoch_year': 2016,
    'first_time_deriv_mean_motion': 2.935e-05,
    'inclination': 51.643,
    'mean_anomaly': 349.8248,
    'mean_motion': 5.54761605,
    'revolution_number_at_epoch': 343,
    'right_ascension_ascending_node': 107.84,
    'satellite_number': u'25544U',
    'second_time_deriv_mean_motion': 0.0}


In [11]:
def AddTLEToDB():
    lines=scrapetle()

    isstle= tletodict(lines)

    import psycopg2
    import sys
    import numpy as np


    con = None

    try:

        con = psycopg2.connect(host='localhost', database='gld', user='asdcadmin', password='ASIMrocks') 
        cur = con.cursor()
        cur.execute("SELECT * FROM isstle LIMIT 0")

        cur.execute("SET TIME ZONE 'UTC';")  # UTC 
        #cur.execute("INSERT INTO isstle ( satellite_number,       epoch_year  ,epoch,                    first_time_deriv_mean_motion   ,second_time_deriv_mean_motion,bstar_drag    ,element_Set_no   ,inclination    ,right_ascension_ascending_node    ,eccentricity    ,argument_perigee    ,mean_anomaly    ,mean_motion   ,revolution_number_at_epoch    )         VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s  )",          ( isstle['satellite_number']   ,  isstle['epoch_year']  ,isstle['epoch'], isstle['first_time_deriv_mean_motion']   ,isstle['second_time_deriv_mean_motion'],isstle['bstar_drag']    ,isstle['element_Set_no']   ,isstle['inclination']    ,isstle['right_ascension_ascending_node']    ,isstle['eccentricity']    ,isstle['argument_perigee']    ,isstle['mean_anomaly']    ,isstle['mean_motion']   ,isstle['revolution_number_at_epoch']           ))       
        #cur.execute("INSERT INTO isstle ( satellite_number,       epoch_year  ,epoch,                    first_time_deriv_mean_motion   ,second_time_deriv_mean_motion,bstar_drag    ,element_Set_no   ,inclination    ,right_ascension_ascending_node    ,eccentricity    ,argument_perigee    ,mean_anomaly    ,mean_motion   ,revolution_number_at_epoch    )         VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s  )",          (             1,1,1,1,1,1,1,1,1,1,1,1,1,1))       
        SQL = "INSERT INTO isstle (satellite_number, epoch_year, epoch,   first_time_deriv_mean_motion, second_time_deriv_mean_motion, bstar_drag,   element_Set_no, inclination,right_ascension_ascending_node,  eccentricity,argument_perigee,mean_anomaly,  mean_motion,revolution_number_at_epoch) VALUES (%s, %s, %s,  %s, %s, %s,     %s, %s, %s,   %s, %s, %s,   %s, %s );" # Note: no quotes
        data = (isstle['satellite_number']   ,  isstle['epoch_year']  ,isstle['epoch'], isstle['first_time_deriv_mean_motion']   ,isstle['second_time_deriv_mean_motion'],isstle['bstar_drag']    ,isstle['element_Set_no']   ,isstle['inclination']    ,isstle['right_ascension_ascending_node']    ,isstle['eccentricity']    ,isstle['argument_perigee']    ,isstle['mean_anomaly']    ,isstle['mean_motion']   ,isstle['revolution_number_at_epoch']  )
        cur.execute(SQL, data) # Note: no % operator
        
        #cur.execute("INSERT INTO isstle (satellite_number, epoch_year) VALUES (200,200);")
        

        cols = isstle.keys()
        vals = [isstle[x] for x in cols]
        vals_str_list = ["%s"] * len(vals)
        vals_str = ", ".join(vals_str_list)

        #cur.execute("INSERT INTO isstle ({cols}) VALUES ({vals_str})".format( cols = cols, vals_str = vals_str), vals)


        con.commit()
    except :
        print ('Error could not connect to database'     )
        #sys.exit(1)


    finally:

        if con:
            con.close()   
    
AddTLEToDB()

5.54761605  3438
